In [1]:
!pip install roboflow

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from ultralytics import YOLO
import os
import cv2
import numpy as np

# download the dataset from roboflow

In [5]:
# download the dataset from roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="QTj9X7XTgxhw26sGOhJp")
project = rf.workspace("myworkspace-l4whl").project("iris_detection-iozew")
dataset = project.version(2).download("yolov8")

# Trainin the yolov8 model

In [6]:
# Trainin the yolov8 model
model = YOLO("yolov8n.pt")  # load a pretrained YOLOv8n model
 
model.train(data="Iris_detection-2/data.yaml")  # train the model
model.val()  # evaluate model performance on the validation set
#model.predict(source="https://ultralytics.com/images/bus.jpg")  # predict on an image
#model.export(format="onnx")  # export the model to ONNX format

In [ ]:
cwd = os.getcwd()

In [ ]:
cwd

# Function for prediction task outputs the predciction results

In [10]:
def detection_img(img_path):
    model_trained = YOLO("runs/detect/train9/weights/best.pt")
    results = model_trained.predict(img_path,save=True)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    return results
    
    

### The predcited image is saved in the following path " runs/detect/predict/image.jpg"

In [11]:
detection_img('data/anchor/RE110.jpg')

Ultralytics YOLOv8.0.25 🚀 Python-3.8.13 torch-1.13.1 CUDA:0 (NVIDIA A100-SXM4-80GB, 81251MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /home/kumar.amit1/ondemand/data/sys/myjobs/projects/default/1/data/data/anchor/RE110.jpg: 640x640 1 Iris, 7.0ms
Speed: 2.1ms pre-process, 7.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([1, 6])
 dtype: torch.float32
  + tensor([[248.00000, 398.00000, 671.00000, 741.00000,   0.90128,   0.00000]], device='cuda:0')]

In [ ]:
model_trained = YOLO("runs/detect/train9/weights/best.pt")

In [ ]:
print(model_trained)

# Creating directiory for preparing data for iris recogniton task 

In [ ]:
# Path for Iris recogniton dataset 
root_folder = 'data'
# Creates and saves path for recogntion task
save_folder = 'data_crop'

In [ ]:
root_folder

In [ ]:
# Define the subfolders to read images from
subfolders = ['anchor', 'negative', 'positive']

# # Create the directory structure
for subfolder in subfolders:
    subfolder_path = os.path.join(root_folder, subfolder)
    print(subfolder_path)
    save_subfolder = subfolder + '_crop'
    save_subfolder_path = os.path.join(save_folder, save_subfolder)
    os.makedirs(save_subfolder_path, exist_ok=True)

In [ ]:
def crop_image(root_folder, save_folder):
    # Define the subfolders to read images from
    subfolders = ['anchor', 'negative', 'positive']

    # Loop through each subfolder
    for subfolder in subfolders:
        subfolder_path = os.path.join(root_folder, subfolder)
        save_subfolder = subfolder + '_crop2'
        save_subfolder_path = os.path.join(save_folder, save_subfolder)
        os.makedirs(save_subfolder_path, exist_ok=True)

        # Loop through each image in the subfolder
        for filename in os.listdir(subfolder_path):
            if filename.endswith('.jpg'):
                image_path = os.path.join(subfolder_path, filename)
                image = cv2.imread(image_path)

                # Perform YOLOv8 prediction
                results = model.predict(image_path)
                for result in results:
                    co = result.boxes.xywh

                # Check if the output contains valid predictions
                if len(co) > 0:
                    x = int(co[0][0])
                    y = int(co[0][1])
                    width = int(co[0][2])
                    height = int(co[0][3])

                    # # Convert to integers
                    # x = int(x)
                    # y = int(y)
                    # width = int(width)
                    # height = int(height)

                    # Calculate the top-left corner coordinates of the bounding box
                    x_min = x - (width // 2)
                    y_min = y - (height // 2)

                    # Calculate the bottom-right corner coordinates of the bounding box
                    x_max = x + (width // 2)
                    y_max = y + (height // 2)

                    # Crop the image using the bounding box coordinates
                    cropped_image = image[y_min:y_max, x_min:x_max]
                    
                    alpha = 1.5  
                    # control brightness by 50
                    beta = 50  
                    image2 = cv2.convertScaleAbs(cropped_image, alpha=alpha, beta=beta)
                    
                    # Apply median blur
                    ksize = 5  # kernel size, should be an odd number
                    image_blur = cv2.medianBlur(image2, ksize)

                    # Create the sharpening kernel
                    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

                    # Sharpen the image
                    sharpened_image = cv2.filter2D(image_blur, -1, kernel)

                    #Save the image
                    cv2.imwrite('sharpened_image.jpg', sharpened_image)

                    # Get the save path for the cropped image
                    save_path = os.path.join(save_subfolder_path, filename)

                    # Save the cropped image to a file
                    cv2.imwrite(save_path, cropped_image)
                else:
                    print(f"No predictions found for {filename}")

# Saves the Enhanced and Cropped image in there respective directory

In [ ]:
crop_image(root_folder, save_folder)